In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import pandas as pd

In [3]:
medicina_alternativa = [
    "Quiropraxia",
    "Hidroterapia",
    "Acupuntura",
    "Musicoterapia",
    "Trofoterapia",
    "Homeopatía",
    "Reflexología",
    "Reiki",

]

medicina_tradicional = [
    "Sobada de cuy",
    "Huesero",
    "Pago con coca",
    "Pasada de huevo",
    "Limpia por curanderos",
    "Pasa o sobada con alumbres",
    "Baños de florecimiento",
    "Uso de barro o arcilla",
]

technique_dict = {technique: "Alternative" for technique in medicina_alternativa}
technique_dict.update({technique: "Traditional" for technique in medicina_tradicional})
technique_dict


{'Quiropraxia': 'Alternative',
 'Hidroterapia': 'Alternative',
 'Acupuntura': 'Alternative',
 'Musicoterapia': 'Alternative',
 'Trofoterapia': 'Alternative',
 'Homeopatía': 'Alternative',
 'Reflexología': 'Alternative',
 'Reiki': 'Alternative',
 'Sobada de cuy': 'Traditional',
 'Huesero': 'Traditional',
 'Pago con coca': 'Traditional',
 'Pasada de huevo': 'Traditional',
 'Limpia por curanderos': 'Traditional',
 'Pasa o sobada con alumbres': 'Traditional',
 'Baños de florecimiento': 'Traditional',
 'Uso de barro o arcilla': 'Traditional'}

In [ ]:
import pandas as pd
from process_text import process_annotations


### Processing reviewed annotations

In [13]:



arguments_df = pd.read_excel('../data/Extracted reasons_revised.xlsx')
arguments_df['technique'] = arguments_df['technique'].apply(lambda s: s.strip())
arguments_df['technique_type'] = arguments_df['technique'].apply(lambda s: technique_dict[s])
arguments_df['extracted_reasons'] = arguments_df['extracted_reasons'].apply(process_annotations)
extracted_reasons = list(set(arguments_df['extracted_reasons'].sum()))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ezequiellopez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[autoreload of process_text failed: Traceback (most recent call last):
  File "/Users/ezequiellopez/_research_projects/__summer/CAM_peru/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 325, in check
    superreload(m, reload, self.old_objects)
  File "/Users/ezequiellopez/_research_projects/__summer/CAM_peru/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 580, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/Users/ezequiellopez/.pyenv/versions/3.12.7/lib/python3.12/importlib/__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  Fi

In [ ]:
df_ = pd.read_excel('../data/classified_micro-reasons_1104_v1.xlsx').drop(columns=['Unnamed: 0'])
df_['categoria'] = df_['categoria'].apply(lambda l: ast.literal_eval(l))
df_['categoria_'] = df_['categoria_'].apply(lambda l: ast.literal_eval(l))
categorized_reasons = list(df_['text'].unique())

df_['categoria'] = df_['categoria'].apply(lambda l: l[0])
df_['categoria_'] = df_['categoria_'].apply(lambda l: l[0])

df_dict = df_.set_index('text')['categoria_'].to_dict()

arguments_df['extracted_reasons'] = arguments_df['extracted_reasons'].apply(lambda l: [c for c in l if len(c)!=0])
arguments_df['associated_classes'] = arguments_df['extracted_reasons'].apply(lambda l: [df_dict[k] for k in l])

arguments_df

In [ ]:
arguments_df['tokenized_response'] = arguments_df['response'].apply(lambda s: preprocess_spanish_text(
    text=s,
    preserve_words={},
    return_debug=False))

arguments_df

### Basic inspection: word clouds

In [ ]:

generate_wordcloud_from_token_column(arguments_df['tokenized_response'], 'Word frequency for all categories (SPA)', max_words=100)
translation = plot_wordcloud_EN(tokenized_column=arguments_df['tokenized_response'], custom_overrides=combined_dict , title="Word frequency for all categories (EN)", n_words=200)


##### wordcloud by technique

In [ ]:
translations = []

for technique in arguments_df['technique_type'].unique():
    _ = arguments_df.query('technique_type == @technique')
    #generate_wordcloud_from_tokens(_['tokenized_response'], f'Word frequency for {technique}')
    generate_wordcloud_from_token_column(_['tokenized_response'], f'Word frequency for {technique} (SPA)', max_words=200)
    translation = plot_wordcloud_EN(tokenized_column=_['tokenized_response'], custom_overrides=combined_dict, title=f"Word frequency for {technique} (EN)", n_words=200)
    translations.append(translation)

### Distribution of reasons per therapy type

In [ ]:
df_exploded = arguments_df.explode('associated_classes')

class_map = {
    "Creencias paranormales":                   "Paranormal beliefs",
    "Desconfianza en la medicina convencional": "Distrust in conventional medicine",
    "Uso de productos naturales":               "Use of natural products",
    "Beneficios personales y efectividad clínica":"Benefits and effectiveness",
    "Testimonios y experiencias personales":     "Testimonies and personal experiences",
    "Tradicionalismo":                          "Traditionalism",
    "Accesibilidad y comodidad":                "Comfort and accessibility",
    "Credenciales científicas y profesionales": "Professional and scientific credentials",
    "Ausencia de efectos secundarios":          "Lack of side effects",
    "Razones imposibles de categorizar":    "Miscellaneous"
}


# Map Spanish categories to English using class_map
df_exploded['associated_classes_EN'] = df_exploded['associated_classes'].replace(class_map)

category_counts_en = df_exploded['associated_classes_EN'].value_counts()

ax = category_counts_en.head(99).plot(kind='barh', figsize=(7, 5), color='skyblue')
plt.title('Number of Occurrences of Extracted Reasons')
plt.xlabel('Count')
plt.ylabel('Category')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('figures/category_counts_barh_EN.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
df_exploded = arguments_df.explode('associated_classes')

category_counts = df_exploded['associated_classes'].value_counts()

ax = category_counts.head(30).plot(kind='barh', figsize=(5, 7), color='skyblue')
plt.title('Number of Occurrences of extracted reasons')
plt.xlabel('Category')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('figures/category_counts_barh.png', dpi=300, bbox_inches='tight')
plt.show()

pivot_table = df_exploded.pivot_table(index='technique', columns='associated_classes', aggfunc='size', fill_value=0)

pivot_table.plot(kind='barh', stacked=True, figsize=(10, 8), colormap='coolwarm')
plt.title('Distribution of reason category by therapy')
plt.xlabel('Count')
plt.ylabel('Tecnica')
plt.legend(title='Categoria_', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# --- your existing data prep for the first two plots ---
df_exploded = arguments_df.explode('associated_classes')

pivot_table = df_exploded.pivot_table(
    index='technique',
    columns='associated_classes',
    aggfunc='size',
    fill_value=0
)
pivot_table = pivot_table.loc[:, pivot_table.columns != '']
normalized_pivot_table = pivot_table.div(pivot_table.sum(axis=1), axis=0)
total_counts = pivot_table.sum(axis=1).sort_values(ascending=True)
pivot_table = pivot_table.loc[total_counts.index]
normalized_pivot_table = normalized_pivot_table.loc[total_counts.index]
total_counts = total_counts.loc[normalized_pivot_table.index]

# --- your third‐plot data prep (reason categories by technique_type) ---
df3 = df_exploded.copy()
pivot3 = df3.pivot_table(
    index='technique_type',
    columns='associated_classes',
    aggfunc='size',
    fill_value=0
)
norm3 = pivot3.div(pivot3.sum(axis=1), axis=0)

# --- set up a 2×3 grid: first two cols for big plots, third col for legend+third plot ---
fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(
    nrows=2,
    ncols=3,
    width_ratios=[3, .8, 2.2],
    height_ratios=[1, 1],
    wspace=0.1,
    hspace=0.4
)

# 1) Normalized stacked bars → col1 (both rows)
ax1 = fig.add_subplot(gs[:, 0])
normalized_pivot_table.plot(
    kind='barh',
    stacked=True,
    ax=ax1,
    colormap='tab20c',
    legend=False
)
ax1.set_title('Normalized Distribution of Associated Classes by Technique')
ax1.set_xlabel('Proportion')
ax1.set_ylabel('Technique')

# 2) Total counts → col2 (both rows), share y-axis
ax2 = fig.add_subplot(gs[:, 1], sharey=ax1)
total_counts.plot(
    kind='barh',
    ax=ax2,
    color='salmon'
)
ax2.set_frame_on(False)
ax2.set_title('Frequency')
ax2.set_xlabel('Count')
ax2.set_ylabel('')

# 3) Legend → top‐right (row1, col3)
legend_ax = fig.add_subplot(gs[0, 2])
legend_ax.axis('off')
h, l = ax1.get_legend_handles_labels()
legend_ax.legend(h, l, title="Reason Type", loc='center')

# 4) Your third plot → bottom‐right (row2, col3)
ax3 = fig.add_subplot(gs[1, 2])
norm3.plot(
    kind='barh',
    stacked=True,
    ax=ax3,
    colormap='tab20c',
    legend=False
)
ax3.set_title('Normalized Distribution \n Reason Categories by Technique Type', pad=10)
ax3.set_xlabel('Proportion')
ax3.set_ylabel('')

plt.tight_layout()
fig.savefig(f'figures/reason_distribution_figure.png', dpi=600, bbox_inches='tight', transparent=False)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1) Mappings
class_map = {
    "Creencias paranormales":                   "Paranormal beliefs",
    "Desconfianza en la medicina convencional": "Distrust in conventional medicine",
    "Uso de productos naturales":               "Use of natural products",
    "Beneficios personales y efectividad clínica":"Benefits and effectiveness",
    "Testimonios y experiencias personales":     "Testimonies and personal experiences",
    "Tradicionalismo":                          "Traditionalism",
    "Accesibilidad y comodidad":                "Comfort and accessibility",
    "Credenciales científicas y profesionales": "Professional and scientific credentials",
    "Ausencia de efectos secundarios":          "Lack of side effects",
    "Razones imposibles de categorizar":        "Miscellaneous"
}

tech_map = {
    "Pasada de huevo":            "(Trad) Egg cleanse",
    "Huesero":                    "(Trad) Bone healer",
    "Pasa o sobada con alumbres": "(Trad) Alum rub",
    "Uso de barro o arcilla":     "(Trad) Use of mud or clay",
    "Limpia por curanderos":      "(Trad) Healer cleanse",
    "Sobada de cuy":              "(Trad) Guinea pig rub",
    "Pago con coca":              "(Trad) Coca payment",
    "Musicoterapia":              "(Alt) Music therapy",
    "Reflexología":               "(Alt) Reflexology",
    "Quiropraxia":                "(Alt) Chiropractic",
    "Acupuntura":                 "(Alt) Acupuncture",
    "Hidroterapia":               "(Alt) Hydrotherapy",
    "Homeopatía":                 "(Alt) Homeopathy",
    "Reiki":                      "(Alt) Reiki",
    "Trofoterapia":               "(Alt) Trophotherapy",
    "Baños de florecimiento":     "(Trad) Flowering baths"
}

# 2) Explode + translate
df_exploded = arguments_df.explode("associated_classes")
df_exploded["associated_classes"] = df_exploded["associated_classes"].replace(class_map)
df_exploded["technique"] = df_exploded["technique"].replace(tech_map)

# 3) Pivot table: counts per technique and category
pivot_table = df_exploded.pivot_table(
    index="technique",
    columns="associated_classes",
    aggfunc="size",
    fill_value=0
)
pivot_table = pivot_table.loc[:, pivot_table.columns != ""]

# 4) Normalize each cell by total sum over all cells (global frequency)
global_sum = pivot_table.values.sum()
freq_weighted = pivot_table / global_sum

# 5) Normalize each row to sum to 1 (so each bar has same length)
row_sums = freq_weighted.sum(axis=1)
scaled_bars = freq_weighted.div(row_sums, axis=0)

# Sort by original frequency for visual clarity
total_counts = pivot_table.sum(axis=1).sort_values(ascending=True)
scaled_bars = scaled_bars.loc[total_counts.index]

# 6) Do the same for technique_type
df3 = df_exploded.copy()
pivot3 = df3.pivot_table(
    index="technique_type",
    columns="associated_classes",
    aggfunc="size",
    fill_value=0
)
pivot3 = pivot3.loc[:, pivot3.columns != ""]
global_sum3 = pivot3.values.sum()
freq_weighted3 = pivot3 / global_sum3
row_sums3 = freq_weighted3.sum(axis=1)
scaled_bars3 = freq_weighted3.div(row_sums3, axis=0)

# 7) Plot
fig = plt.figure(figsize=(14, 8))
gs = fig.add_gridspec(
    nrows=2, ncols=3,
    width_ratios=[3, 0.8, 2.2],
    height_ratios=[1, 1],
    wspace=0.1, hspace=0.4
)

# Left plot — technique level, frequency-weighted & row-normalized
ax1 = fig.add_subplot(gs[:, 0])
scaled_bars.plot(
    kind='barh', stacked=True, ax=ax1, colormap='Set3', legend=False
)
ax1.set_title('Normalized Bar (Equal Size),\nProportions Weighted by Global Frequency')
ax1.set_xlabel('Proportion')
ax1.set_ylabel('Technique')

# Middle plot — raw frequency
ax2 = fig.add_subplot(gs[:, 1], sharey=ax1)
total_counts.plot(kind='barh', ax=ax2, color='salmon')
ax2.set_frame_on(False)
ax2.set_title('Frequency')
ax2.set_xlabel('Count')
ax2.set_ylabel('')

# Legend — top right
legend_ax = fig.add_subplot(gs[0, 2])
legend_ax.axis('off')
handles, labels = ax1.get_legend_handles_labels()
legend_ax.legend(handles, labels, title="Reason Category", loc='center')

# Bottom right plot — technique type (same logic)
ax3 = fig.add_subplot(gs[1, 2])
scaled_bars3.plot(
    kind='barh', stacked=True, ax=ax3, colormap='Set3', legend=False
)
ax3.set_title('Technique Type: Normalized Bar\nwith Globally Weighted Proportions')
ax3.set_xlabel('Proportion')
ax3.set_ylabel('')

# Finalize
plt.tight_layout()
fig.savefig('figures/reason_distribution_figure__EN_weighted_scaled.png', dpi=600, bbox_inches='tight', transparent=False)
plt.show()


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# ---- 1. Compute cosine similarity matrix ----
similarity_matrix = pd.DataFrame(
    cosine_similarity(scaled_bars),
    index=scaled_bars.index,
    columns=scaled_bars.index
)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.patches as mpatches

# Clean label function
def clean_label(label):
    return label.replace('(Trad) ', '').replace('(Alt) ', '')

# Build graph
G = nx.Graph()
for technique in similarity_matrix.index:
    G.add_node(technique, display=clean_label(technique))

threshold = 0.9
for i in similarity_matrix.index:
    for j in similarity_matrix.columns:
        if i != j and similarity_matrix.loc[i, j] >= threshold:
            G.add_edge(i, j, weight=similarity_matrix.loc[i, j])

# Node types and colors
node_color_map = {'Alternative': 'skyblue', 'Traditional': 'salmon'}
node_types = ['Alternative' if '(Alt)' in n else 'Traditional' for n in G.nodes()]
node_colors = [node_color_map[t] for t in node_types]

# Layout: More spacing
pos = nx.spring_layout(G, seed=585, k=1, iterations=100)

# Font and style tuning
plt.rcParams.update({
    "font.family": "Arial",
    "font.size": 12,
    "axes.titlesize": 14,
    "legend.fontsize": 11,
    "legend.title_fontsize": 12
})

plt.figure(figsize=(12, 9))

# Draw nodes
nx.draw_networkx_nodes(
    G, pos,
    node_color=node_colors,
    node_size=1000,
    edgecolors='black',
    linewidths=0.8
)

# Draw soft edges
# edges = G.edges(data=True)
# weights = [d['weight'] for _, _, d in edges]
# nx.draw_networkx_edges(
#     G, pos,
#     edge_color='gray',
#     width=[(w - threshold) * 20 for w in weights],
#     alpha=0.2
# )

edges = G.edges(data=True)
weights = [d['weight'] for _, _, d in edges]
nx.draw_networkx_edges(G, pos, edge_color='gray', width=[(w - threshold) * 20 for w in weights])

# Draw labels with white background for readability
labels = {n: G.nodes[n]['display'] for n in G.nodes()}
nx.draw_networkx_labels(
    G, pos,
    labels=labels,
    font_size=12,
    bbox=dict(facecolor='white', edgecolor='none', boxstyle='round,pad=0.1', alpha=0.1)
)

# Legend
legend_handles = [
    mpatches.Patch(color='skyblue', label='Alternative'),
    mpatches.Patch(color='salmon', label='Traditional')
]
plt.legend(
    handles=legend_handles,
    title="Technique Type",
    loc='upper right',
    frameon=True,
    edgecolor='black'
)

# Clean display
plt.axis('off')
plt.tight_layout()
plt.savefig("figures/technique_similarity_graph_cleaned.png", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.stats import binomtest
import os
import matplotlib.patches as patches

# === 1. Setup: Label mappings ===
class_map = {
    "Creencias paranormales": "Paranormal beliefs",
    "Desconfianza en la medicina convencional": "Distrust in conventional medicine",
    "Uso de productos naturales": "Use of natural products",
    "Beneficios personales y efectividad clínica": "Benefits and effectiveness",
    "Testimonios y experiencias personales": "Testimonies and personal experiences",
    "Tradicionalismo": "Traditionalism",
    "Accesibilidad y comodidad": "Comfort and accessibility",
    "Credenciales científicas y profesionales": "Professional and scientific credentials",
    "Ausencia de efectos secundarios": "Lack of side effects",
    "Razones imposibles de categorizar": "Miscellaneous"
}

tech_map = {
    "Traditional": "Traditional",
    "Alternative": "Alternative",
    "Tradicional": "Traditional",
    "Alternativa": "Alternative"
}

# === 2. Data preparation ===
df = arguments_df.dropna(subset=["associated_classes"]).copy()
df["associated_classes"] = df["associated_classes"].apply(lambda x: eval(x) if isinstance(x, str) else x)
df["associated_classes"] = df["associated_classes"].apply(lambda lst: [class_map.get(c, c) for c in lst])
df["technique_type"] = df["technique_type"].replace(tech_map)

# === 3. Build reason pair dataset (including diagonals) ===
pair_tech_counts = []
for _, row in df.iterrows():
    tech = row["technique_type"]
    classes = sorted(set(row["associated_classes"]))
    if len(classes) == 1:
        a = classes[0]
        pair_tech_counts.append({"pair": (a, a), "technique_type": tech})
    else:
        for a, b in combinations(classes, 2):
            pair_tech_counts.append({"pair": (a, b), "technique_type": tech})

pair_df = pd.DataFrame(pair_tech_counts)

# === 4. Compute probabilities ===
pair_counts = pair_df.groupby(["pair", "technique_type"]).size().unstack(fill_value=0)
pair_counts["total"] = pair_counts.sum(axis=1)
pair_counts["p_traditional"] = pair_counts.get("Traditional", 0) / pair_counts["total"]

# Drop very rare pairs
pair_counts = pair_counts[pair_counts["total"] >= 5]

# === 5. Compute p-values ===
pair_counts["p_value"] = pair_counts.apply(
    lambda row: binomtest(int(row.get("Traditional", 0)), int(row["total"]), p=0.5, alternative="two-sided").pvalue,
    axis=1
)
pair_counts["significant"] = pair_counts["p_value"] < 0.05

# === 6. Build matrices ===
pairs = pair_counts.index.tolist()
all_reasons = sorted(set(r for pair in pairs for r in pair))

matrix = pd.DataFrame(index=all_reasons, columns=all_reasons, dtype=float)
sig_matrix = pd.DataFrame(index=all_reasons, columns=all_reasons, dtype=bool)

for (a, b), row in pair_counts.iterrows():
    p = row["p_traditional"]
    sig = row["significant"]
    matrix.loc[a, b] = p
    matrix.loc[b, a] = p
    sig_matrix.loc[a, b] = sig
    sig_matrix.loc[b, a] = sig

# === 7. Cluster order ===
filled_matrix = matrix.fillna(0)
link = linkage(filled_matrix, method='average', metric='cosine')
order = leaves_list(link)
ordered_reasons = [matrix.index[i] for i in order]
matrix = matrix.loc[ordered_reasons, ordered_reasons]
sig_matrix = sig_matrix.loc[ordered_reasons, ordered_reasons]

# === 8. Rescale ===
rescaled_matrix = 2 * (matrix - 0.5)

# === 9. Annotations ===
annot_matrix = pd.DataFrame(index=matrix.index, columns=matrix.columns, dtype=object)
for (a, b), row in pair_counts.iterrows():
    rescaled_val = 2 * (row["p_traditional"] - 0.5)
    is_sig = row["significant"]
    # label = f"{rescaled_val:+.2f}" + ("*" if is_sig else "")
    if rescaled_val == 0:
        label = "0.00" + ("*" if is_sig else "")
    else:
        label = f"{rescaled_val:+.2f}" + ("*" if is_sig else "")
    
    annot_matrix.loc[a, b] = label
    annot_matrix.loc[b, a] = label

# === Fix: Fill "NA" where data is missing in matrix, and set rescaled_matrix to 0 to show it
for i in matrix.index:
    for j in matrix.columns:
        if pd.isna(matrix.loc[i, j]):
            annot_matrix.loc[i, j] = "NA"
            rescaled_matrix.loc[i, j] = 0  # neutral color so it renders

# === 10. Plot ===
mask = np.triu(np.ones_like(rescaled_matrix, dtype=bool), k=1)

plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    rescaled_matrix,
    annot=annot_matrix,
    fmt="",
    cmap="RdBu_r",
    vmin=-1, vmax=1,
    center=0,
    square=True,
    linewidths=0,  # no seaborn grid
    linecolor='gray',
    cbar_kws={
        "label": "← Alternative       |       Equal       |       Traditional →",
        "ticks": [-1, -0.5, 0, 0.5, 1]
    },
    mask=mask
)

# === Add manual gridlines for lower triangle
n = len(rescaled_matrix)
for i in range(n):
    for j in range(i + 1):
        if not pd.isna(rescaled_matrix.iloc[i, j]):
            rect = patches.Rectangle((j, i), 1, 1, fill=False, edgecolor='gray', linewidth=0.5)
            ax.add_patch(rect)

#plt.title("Technique Type Likelihood (Rescaled, Centered at 0)\nGiven Co-occurrence of Reasons Pairs", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

# === 11. Save ===
os.makedirs("figures", exist_ok=True)
plt.savefig("figures/reason_pair_traditional_probability_heatmap_with_diagonal.png", dpi=600, bbox_inches="tight")
plt.show()
